In [2]:
import pandas as pd
import numpy as np
from datetime import datetime

df = pd.read_csv("../dataset/train.csv")
df.head()

,Unnamed: 0,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price
0,1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,Diesel,Manual,First,19.67 kmpl,1582 CC,126.2 bhp,5.0,NaN,12.50
1,2,Honda Jazz V,Chennai,2011,46000,Petrol,Manual,First,13 km/kg,1199 CC,88.7 bhp,5.0,8.61 Lakh,4.50
2,3,Maruti Ertiga VDI,Chennai,2012,87000,Diesel,Manual,First,20.77 kmpl,1248 CC,88.76 bhp,7.0,NaN,6.00
3,4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Diesel,Automatic,Second,15.2 kmpl,1968 CC,140.8 bhp,5.0,NaN,17.74
4,6,Nissan Micra Diesel XV,Jaipur,2013,86999,Diesel,Manual,First,23.08 kmpl,1461 CC,63.1 bhp,5.0,NaN,3.50


A

In this step I just checked the basic structure of the dataset and how many missing values each column has. This helps me decide which columns to impute and which ones to drop.

In [3]:
df.shape

(5847, 14)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5847 entries, 0 to 5846
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         5847 non-null   int64  
 1   Name               5847 non-null   object 
 2   Location           5847 non-null   object 
 3   Year               5847 non-null   int64  
 4   Kilometers_Driven  5847 non-null   int64  
 5   Fuel_Type          5847 non-null   object 
 6   Transmission       5847 non-null   object 
 7   Owner_Type         5847 non-null   object 
 8   Mileage            5845 non-null   object 
 9   Engine             5811 non-null   object 
 10  Power              5811 non-null   object 
 11  Seats              5809 non-null   float64
 12  New_Price          815 non-null    object 
 13  Price              5847 non-null   float64
dtypes: float64(2), int64(3), object(9)
memory usage: 639.6+ KB


In [5]:
df.isna().sum()

Unnamed: 0              0
Name                    0
Location                0
Year                    0
Kilometers_Driven       0
Fuel_Type               0
Transmission            0
Owner_Type              0
Mileage                 2
Engine                 36
Power                  36
Seats                  38
New_Price            5032
Price                   0
dtype: int64

From the missing value counts:
	•	New_Price has a lot of missing values (majority of rows). Imputing this would be mostly guessing, so I decided to drop this column.
	•	Mileage, Engine, Power, and Seats have relatively fewer missing values. After converting them to numeric, I will fill their missing values using the median, since these are continuous variables and the median is more robust to outliers than the mean.

B

Here I removed units like kmpl, CC, bhp, and Lakh and kept only the numeric part. I created new numeric columns for these.

In [6]:
df_clean = df.copy()

import re

def extract_number(x):
    if pd.isna(x):
        return np.nan
    x = str(x)
    nums = re.findall(r"[\d\.]+", x)
    if len(nums) == 0:
        return np.nan
    return float(nums[0])

df_clean["Mileage_num"] = df_clean["Mileage"].apply(extract_number)
df_clean["Engine_num"] = df_clean["Engine"].apply(extract_number)
df_clean["Power_num"] = df_clean["Power"].apply(extract_number)
df_clean["New_Price_num"] = df_clean["New_Price"].apply(extract_number)

df_clean[["Mileage", "Mileage_num",
          "Engine", "Engine_num",
          "Power", "Power_num",
          "New_Price", "New_Price_num"]].head()

,Mileage,Mileage_num,Engine,Engine_num,Power,Power_num,New_Price,New_Price_num
0,19.67 kmpl,19.67,1582 CC,1582.0,126.2 bhp,126.20,NaN,NaN
1,13 km/kg,13.00,1199 CC,1199.0,88.7 bhp,88.70,8.61 Lakh,8.61
2,20.77 kmpl,20.77,1248 CC,1248.0,88.76 bhp,88.76,NaN,NaN
3,15.2 kmpl,15.20,1968 CC,1968.0,140.8 bhp,140.80,NaN,NaN
4,23.08 kmpl,23.08,1461 CC,1461.0,63.1 bhp,63.10,NaN,NaN


In [7]:
df_clean = df_clean.drop(columns=["Mileage", "Engine", "Power", "New_Price"])
df_clean.head()

,Unnamed: 0,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Seats,Price,Mileage_num,Engine_num,Power_num,New_Price_num
0,1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,Diesel,Manual,First,5.0,12.50,19.67,1582.0,126.20,NaN
1,2,Honda Jazz V,Chennai,2011,46000,Petrol,Manual,First,5.0,4.50,13.00,1199.0,88.70,8.61
2,3,Maruti Ertiga VDI,Chennai,2012,87000,Diesel,Manual,First,7.0,6.00,20.77,1248.0,88.76,NaN
3,4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Diesel,Automatic,Second,5.0,17.74,15.20,1968.0,140.80,NaN
4,6,Nissan Micra Diesel XV,Jaipur,2013,86999,Diesel,Manual,First,5.0,3.50,23.08,1461.0,63.10,NaN


Now I drop the numeric New_Price_num column (still too many missing values) and impute the remaining numeric columns with their median values.

In [8]:
# Drop New_Price_num because it has too many missing values
df_clean = df_clean.drop(columns=["New_Price_num"])

# Impute median for numeric columns with a few missing values
for col in ["Mileage_num", "Engine_num", "Power_num", "Seats"]:
    med = df_clean[col].median()
    df_clean[col] = df_clean[col].fillna(med)

df_clean[["Mileage_num", "Engine_num", "Power_num", "Seats"]].isna().sum()

Mileage_num    0
Engine_num     0
Power_num      0
Seats          0
dtype: int64

C

Next, I converted the categorical variables Fuel_Type and Transmission into one-hot encoded numeric columns.

In [9]:
df_encoded = pd.get_dummies(
    df_clean,
    columns=["Fuel_Type", "Transmission"],
    drop_first=False
)

df_encoded.head()

,Unnamed: 0,Name,Location,Year,Kilometers_Driven,Owner_Type,Seats,Price,Mileage_num,Engine_num,Power_num,Fuel_Type_Diesel,Fuel_Type_Electric,Fuel_Type_Petrol,Transmission_Automatic,Transmission_Manual
0,1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,First,5.0,12.50,19.67,1582.0,126.20,True,False,False,False,True
1,2,Honda Jazz V,Chennai,2011,46000,First,5.0,4.50,13.00,1199.0,88.70,False,False,True,False,True
2,3,Maruti Ertiga VDI,Chennai,2012,87000,First,7.0,6.00,20.77,1248.0,88.76,True,False,False,False,True
3,4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Second,5.0,17.74,15.20,1968.0,140.80,True,False,False,True,False
4,6,Nissan Micra Diesel XV,Jaipur,2013,86999,First,5.0,3.50,23.08,1461.0,63.10,True,False,False,False,True


D

I created a new feature called Car_Age, which is simply the current year minus the year of manufacture of the car.

In [10]:
current_year = datetime.now().year  # e.g., 2025

df_encoded = df_encoded.assign(
    Car_Age = current_year - df_encoded["Year"]
)

df_encoded[["Year", "Car_Age"]].head()

,Year,Car_Age
0,2015,10
1,2011,14
2,2012,13
3,2013,12
4,2013,12


E

Select: here I picked only a few columns like car name, location, age, and price.

In [11]:
df_select = df_encoded[["Name", "Location", "Car_Age", "Price"]]
df_select.head()

,Name,Location,Car_Age,Price
0,Hyundai Creta 1.6 CRDi SX Option,Pune,10,12.50
1,Honda Jazz V,Chennai,14,4.50
2,Maruti Ertiga VDI,Chennai,13,6.00
3,Audi A4 New 2.0 TDI Multitronic,Coimbatore,12,17.74
4,Nissan Micra Diesel XV,Jaipur,12,3.50


Filter: here I filtered cars that are in Mumbai and have price greater than 10 lakhs.

In [12]:
df_filter = df_encoded[
    (df_encoded["Location"] == "Mumbai") &
    (df_encoded["Price"] > 10)
]

df_filter[["Name", "Location", "Price"]].head()

,Name,Location,Price
5,Toyota Innova Crysta 2.8 GX AT 8S,Mumbai,17.50
20,Audi A6 2011-2015 35 TFSI Technology,Mumbai,23.50
43,Toyota Corolla Altis G AT,Mumbai,10.11
58,Toyota Innova Crysta 2.4 GX MT 8S,Mumbai,16.50
67,Hyundai Creta 1.4 E Plus Diesel,Mumbai,10.50


Rename: I renamed Kilometers_Driven to Odometer_km for clarity.

In [13]:
df_rename = df_encoded.rename(columns={"Kilometers_Driven": "Odometer_km"})
df_rename[["Odometer_km"]].head()

,Odometer_km
0,41000
1,46000
2,87000
3,40670
4,86999


Mutate: I created a new feature Price_per_10k_km, which is the price per 10,000 km driven.

In [14]:
df_mutate = df_encoded.assign(
    Price_per_10k_km = df_encoded["Price"] / (df_encoded["Kilometers_Driven"] / 10000)
)

df_mutate[["Price", "Kilometers_Driven", "Price_per_10k_km"]].head()

,Price,Kilometers_Driven,Price_per_10k_km
0,12.50,41000,3.048780
1,4.50,46000,0.978261
2,6.00,87000,0.689655
3,17.74,40670,4.361938
4,3.50,86999,0.402303


Arrange: I sorted the cars in descending order of price.

In [15]:
df_arrange = df_encoded.sort_values(by="Price", ascending=False)
df_arrange[["Name", "Location", "Price"]].head()

,Name,Location,Price
3952,Land Rover Range Rover 3.0 Diesel LWB Vogue,Hyderabad,160.00
5620,Lamborghini Gallardo Coupe,Delhi,120.00
5752,Jaguar F Type 5.0 V8 S,Hyderabad,100.00
1457,Land Rover Range Rover Sport SE,Kochi,97.07
1917,BMW 7 Series 740Li,Coimbatore,93.67


Summarize + group_by: I grouped the data by Fuel_Type and Transmission (using the original cleaned DataFrame before one-hot encoding) and calculated average price, average mileage, and the count of cars in each group.

In [16]:
# For this, use df_clean where Fuel_Type and Transmission still exist
summary_by_fuel_trans = (
    df_clean
    .groupby(["Fuel_Type", "Transmission"])
    .agg(
        avg_price   = ("Price", "mean"),
        avg_mileage = ("Mileage_num", "mean"),
        count       = ("Price", "size")
    )
    .reset_index()
)

summary_by_fuel_trans

,Fuel_Type,Transmission,avg_price,avg_mileage,count
0,Diesel,Automatic,24.618490,15.711338,1106
1,Diesel,Manual,6.686462,20.235679,2055
2,Electric,Automatic,12.875000,18.190000,2
3,Petrol,Automatic,11.252450,15.812020,604
4,Petrol,Manual,4.160803,18.088889,2080
